In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from geopandas import GeoDataFrame, read_file
# from geopandas.tools import sjoin
from shapely.geometry import Point, mapping,shape
import time

In [2]:
from shapely import prepared


def sjoin(left_df, right_df, how='inner', op='intersects',
          lsuffix='left', rsuffix='right', **kwargs):
    """Spatial join of two GeoDataFrames.
    left_df, right_df are GeoDataFrames
    how: type of join
        left -> use keys from left_df; retain only left_df geometry column
        right -> use keys from right_df; retain only right_df geometry column
        inner -> use intersection of keys from both dfs;
                 retain only left_df geometry column
    op: binary predicate {'intersects', 'contains', 'within'}
        see http://toblerity.org/shapely/manual.html#binary-predicates
    lsuffix: suffix to apply to overlapping column names (left GeoDataFrame)
    rsuffix: suffix to apply to overlapping column names (right GeoDataFrame)
    """

    import rtree

    allowed_hows = ['left', 'right', 'inner']
    if how not in allowed_hows:
        raise ValueError("`how` was \"%s\" but is expected to be in %s" % \
            (how, allowed_hows))

    allowed_ops = ['contains', 'within', 'intersects']
    if op not in allowed_ops:
        raise ValueError("`op` was \"%s\" but is expected to be in %s" % \
            (op, allowed_ops))

    if op == "within":
        # within implemented as the inverse of contains; swap names
        left_df, right_df = right_df, left_df

    if left_df.crs != right_df.crs:
        print('Warning: CRS does not match!')

    tree_idx = rtree.index.Index()
    right_df_bounds = right_df['geometry'].apply(lambda x: x.bounds)
    for i in right_df_bounds.index:
        tree_idx.insert(i, right_df_bounds[i])

    idxmatch = (left_df['geometry'].apply(lambda x: x.bounds)
                .apply(lambda x: list(tree_idx.intersection(x))))
    idxmatch = idxmatch[idxmatch.apply(len) > 0]

    r_idx = np.concatenate(idxmatch.values)
    l_idx = np.concatenate([[i] * len(v) for i, v in idxmatch.iteritems()])

    # Vectorize predicate operations
    def find_intersects(a1, a2):
        return a1.intersects(a2)

    def find_contains(a1, a2):
        return a1.contains(a2)

    predicate_d = {'intersects': find_intersects,
                   'contains': find_contains,
                   'within': find_contains}

    check_predicates = np.vectorize(predicate_d[op])

    result = (
              pd.DataFrame(
                  np.column_stack(
                      [l_idx,
                       r_idx,
                       check_predicates(
                           left_df['geometry']
                           .apply(lambda x: prepared.prep(x))[l_idx],
                           right_df['geometry'][r_idx])
                       ]))
               )

    result.columns = ['index_%s' % lsuffix, 'index_%s' % rsuffix, 'match_bool']
    result = (
              pd.DataFrame(result[result['match_bool']==1])
              .drop('match_bool', axis=1)
              )

    if op == "within":
        # within implemented as the inverse of contains; swap names
        left_df, right_df = right_df, left_df
        result = result.rename(columns={
                    'index_%s' % (lsuffix): 'index_%s' % (rsuffix),
                    'index_%s' % (rsuffix): 'index_%s' % (lsuffix)})

    if how == 'inner':
        result = result.set_index('index_%s' % lsuffix)
        return (
                left_df
                .merge(result, left_index=True, right_index=True)
                .merge(right_df.drop('geometry', axis=1),
                    left_on='index_%s' % rsuffix, right_index=True,
                    suffixes=('_%s' % lsuffix, '_%s' % rsuffix))
                )
    elif how == 'left':
        result = result.set_index('index_%s' % lsuffix)
        return (
                left_df
                .merge(result, left_index=True, right_index=True, how='left')
                .merge(right_df.drop('geometry', axis=1),
                    how='left', left_on='index_%s' % rsuffix, right_index=True,
                    suffixes=('_%s' % lsuffix, '_%s' % rsuffix))
                )
    elif how == 'right':
        return (
                left_df
                .drop('geometry', axis=1)
                .merge(result.merge(right_df,
                    left_on='index_%s' % rsuffix, right_index=True,
                    how='right'), left_index=True,
                    right_on='index_%s' % lsuffix, how='right')
                .set_index('index_%s' % rsuffix)
                )

In [3]:
shapefile='data/cb_2014_us_zcta510_500k.shp'
df=pd.read_csv('data/jfk_sep14.csv',index_col=None)#,nrows=50000)# Limit to 20 rows for testing    

In [4]:
df.head()

,vendor_id,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,rate_code,passenger_count,trip_distance,payment_type,fare_amount,extra,mta_tax,imp_surcharge,tip_amount,tolls_amount,total_amount,store_and_fwd_flag
0,VTS,2014-09-23 10:08:00 UTC,2014-09-23 10:19:00 UTC,-73.781895,40.644637,-73.717507,40.691610,1,2,6.80,CRD,20,0.0,0.5,NaN,2.00,0.00,22.50,NaN
1,VTS,2014-09-23 11:36:00 UTC,2014-09-23 12:11:00 UTC,-73.789865,40.646870,-73.867097,40.768817,1,1,11.82,CRD,38,0.0,0.5,NaN,7.60,0.00,46.10,NaN
2,CMT,2014-09-03 08:56:30 UTC,2014-09-03 09:49:43 UTC,-73.776837,40.645081,-73.974240,40.793701,2,2,20.40,CRD,52,0.0,0.5,NaN,11.56,5.33,69.39,N
3,CMT,2014-09-03 00:36:04 UTC,2014-09-03 00:53:58 UTC,-73.783408,40.648988,-73.923733,40.761242,1,1,14.10,CRD,38,0.5,0.5,NaN,7.80,0.00,46.80,N
4,CMT,2014-09-02 21:10:53 UTC,2014-09-02 21:55:36 UTC,-73.790242,40.646980,-74.003167,40.739118,2,1,17.70,CRD,52,0.0,0.5,NaN,9.00,5.33,66.83,N


In [5]:
if __name__=="__main__":
    start=time.time()
    df['geometry'] = df.apply(lambda z: Point(z.dropoff_longitude, z.dropoff_latitude), axis=1)
    PointsGeodataframe = gpd.GeoDataFrame(df)
#    GeoDataFrame.from_file(shapefile)
    PolygonsGeodataframe = gpd.GeoDataFrame.from_file(shapefile)
    PointsGeodataframe.crs = PolygonsGeodataframe.crs
    print (time.time()-start)
    merged=sjoin(PointsGeodataframe, PolygonsGeodataframe, op='within', how='left')
    print (time.time()-start)
    merged.to_csv('output.csv',index=None)
    print (time.time()-start)

IndexError: list index out of range

In [19]:
df['passenger_count'].sum()

438734L

In [6]:
merged.groupby('GEOID10').size().to_csv('sep14_yellow_by_zip.csv')

NameError: name 'merged' is not defined

In [7]:
uber = pd.read_csv('data/uber-raw-data-sep14.csv')
uber = uber[uber['Lat']<40.651381]
uber = uber[uber['Lat']>40.640668]
uber = uber[uber['Lon']<-73.776283]
uber = uber[uber['Lon']>-73.794694]
uber.shape

(23268, 4)

In [9]:
df2 = pd.read_csv('data/American_B01362.csv')
df2 = df2[df2['DATE']>='9/1/2014']

,DATE,TIME,PICK UP ADDRESS,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,7/1/2014,12:00:00 AM,"874 E 139th St Mott Haven, BX",NaN,NaN,NaN
1,7/1/2014,12:01:00 AM,"628 E 141st St Mott Haven, BX",NaN,NaN,NaN
2,7/1/2014,12:01:00 AM,"601 E 156th St South Bronx, BX",NaN,NaN,NaN
3,7/1/2014,12:01:00 AM,"708 E 138th St Mott Haven, BX",NaN,NaN,NaN
4,7/1/2014,12:02:00 AM,"700 E 140th St Mott Haven, BX",NaN,NaN,NaN


In [10]:
df3 = pd.read_csv('data/Carmel_B00256.csv')
df3 = df3[df3['Date']>='9/1/2014']
df3[df3['PU_Adress']=='JFK']

,Date,Time,PU_Adress,Base_No
170724,9/1/2014,0:00,JFK,B00256
170729,9/1/2014,0:05,JFK,B00256
170730,9/1/2014,0:05,JFK,B00256
170740,9/1/2014,0:15,JFK,B00256
170746,9/1/2014,0:24,JFK,B00256
170748,9/1/2014,0:25,JFK,B00256
170749,9/1/2014,0:26,JFK,B00256
170751,9/1/2014,0:29,JFK,B00256
170756,9/1/2014,0:33,JFK,B00256
170757,9/1/2014,0:36,JFK,B00256


In [14]:
df4 = pd.read_csv('data/Dial7_B00887.csv')
df4 = df4[df4['Date']>='2014.09.01']
df4[df4['State'].str[0:3]=='JFK']

,Date,Time,State,PuFrom,Address,Street
184,2014.09.07,7:52,JFK EL AL Pickup Area C ...,NaN,,
207,2014.09.01,5:59,JFK JET BLUE DOMESTIC PICK UP AREA 3 ...,NaN,,
228,2014.09.15,7:05,JFK JET BLUE DOMESTIC PICK UP AREA 3 ...,NaN,,
265,2014.09.21,8:29,JFK AIR INDIA Pickup Area C ...,NaN,,
274,2014.09.24,8:33,JFK JET BLUE DOMESTIC PICK UP AREA 3 ...,NaN,,
282,2014.09.27,7:15,JFK DELTA TERM 4 P/U AREA C ...,NaN,,
293,2014.09.28,8:00,JFK DELTA TERM 2 CURBSIDE ...,NaN,,
581,2014.09.15,12:54,JFK JET BLUE DOMESTIC PICK UP AREA 3 ...,NaN,,
612,2014.09.11,12:16,JFK AMERICAN DOMESTIC Pickup Area C ...,NaN,,
616,2014.09.08,7:22,JFK Virgin America Pickup Area C ...,JFK,,
